In [1]:
!nvidia-smi

Sun Aug  4 17:10:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 🧪 Install ultralytics roboflow

In [3]:
!pip install roboflow \
ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [20]:
# Чтобы не лагала консоль

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
from roboflow import Roboflow

VER = 1
PROJECT_NAME = "person_detection_alrs"

rf = Roboflow(api_key="u7BX4sli0NcjDQaLoDoz")
project = rf.workspace("alrosa").project(PROJECT_NAME)
version = project.version(VER)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.72, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to person_detection_alrs-1 in yolov8:: 100%|██████████| 1488/1488 [00:00<00:00, 4615.02it/s]


In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [6]:
import yaml

# Открытие файла data.yaml для чтения
with open(f'{HOME}/{PROJECT_NAME}-{VER}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Изменение путей для ключей "train" и "val"
data['train'] = f'{HOME}/{PROJECT_NAME}-{VER}/train/images'
data['val'] = f'{HOME}/{PROJECT_NAME}-{VER}/valid/images'

# Запись изменений обратно в файл
with open(f'{HOME}/{PROJECT_NAME}-{VER}/data.yaml', 'w') as file:
    yaml.dump(data, file)

In [19]:
!ls {PROJECT_NAME}-{VER}/

data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [8]:
os.environ['WANDB_DISABLED'] = 'true'

In [9]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data=f'{HOME}/{PROJECT_NAME}-{VER}/data.yaml',
                      epochs=50,
                      imgsz=640,
                      batch=80,
                      device=[0])

100%|██████████| 6.25M/6.25M [00:00<00:00, 130MB/s]


Ultralytics YOLOv8.2.72 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/person_detection_alrs-1/data.yaml, epochs=50, time=None, patience=100, batch=80, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

100%|██████████| 755k/755k [00:00<00:00, 26.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/person_detection_alrs-1/train/labels... 443 images, 20 backgrounds, 0 corrupt: 100%|██████████| 443/443 [00:00<00:00, 1794.35it/s]

...

      50/50      10.3G     0.5887     0.5417      1.058         83        640: 100%|██████████| 6/6 [00:04<00:00,  1.37it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.86s/it]

                   all        148        277      0.789      0.794      0.837      0.551



50 epochs completed in 0.136 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.72 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


                   all        148        277       0.79      0.794      0.838      0.551
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train


In [10]:
# Валидация
val_results = model.val(data=f'{HOME}/{PROJECT_NAME}-{VER}/data.yaml')

# Вывод результатов валидации
print("Результаты валидации:")
print(f"Precision: {val_results.box.map50:.4f}")
print(f"Recall: {val_results.box.map50:.4f}")
print(f"mAP@50: {val_results.box.map:.4f}")
print(f"mAP@50-95: {val_results.box.map:.4f}")

Ultralytics YOLOv8.2.72 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/person_detection_alrs-1/valid/labels.cache... 148 images, 4 backgrounds, 0 corrupt: 100%|██████████| 148/148 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


                   all        148        277      0.789      0.794      0.836      0.551
Speed: 0.3ms preprocess, 5.0ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train2
Результаты валидации:
Precision: 0.8363
Recall: 0.8363
mAP@50: 0.5511
mAP@50-95: 0.5511


In [21]:
# Архивируем и качаем

import datetime
from IPython.display import FileLink

# Получение текущей даты и времени
current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M")

# Формирование имени архива с текущей датой и временем
archive_name = f"runs_{current_time}.zip"

# Архивирование папки
!tar -czvf {HOME}/{archive_name} -C {HOME} runs

# Ссылка для скачивания архива
FileLink(f"{archive_name}")

runs/
runs/detect/
runs/detect/train2/
runs/detect/train2/confusion_matrix.png
runs/detect/train2/F1_curve.png
runs/detect/train2/P_curve.png
runs/detect/train2/R_curve.png
runs/detect/train2/val_batch1_pred.jpg
runs/detect/train2/confusion_matrix_normalized.png
runs/detect/train2/val_batch0_pred.jpg
runs/detect/train2/val_batch0_labels.jpg
runs/detect/train2/val_batch1_labels.jpg
runs/detect/train2/PR_curve.png
runs/detect/train/
runs/detect/train/labels_correlogram.jpg
runs/detect/train/results.png
runs/detect/train/confusion_matrix.png
runs/detect/train/labels.jpg
runs/detect/train/results.csv
runs/detect/train/F1_curve.png
runs/detect/train/train_batch241.jpg
runs/detect/train/P_curve.png
runs/detect/train/train_batch240.jpg
runs/detect/train/R_curve.png
runs/detect/train/confusion_matrix_normalized.png
runs/detect/train/train_batch0.jpg
runs/detect/train/val_batch0_pred.jpg
runs/detect/train/train_batch242.jpg
runs/detect/train/args.yaml
runs/detect/train/events.out.tfevents.17227

/content/runs_20240804_1724.zip

In [ ]:
# Передаем моедль в roboflow
version.deploy(model_type='yolov8',
               model_path=f'{HOME}/runs/detect/train/')

Dependency ultralytics==8.0.196 is required but found version=8.2.71, to fix: `pip install ultralytics==8.0.196`


Would you like to continue with the wrong version of ultralytics? y/n:  y


View the status of your deployment at: https://app.roboflow.com/alrosa/person_detection_alrs/1
Share your model with the world at: https://universe.roboflow.com/alrosa/person_detection_alrs/model/1
